# Assignment 3: Build a seq2seq model for machine translation.

### Name: [Ayomide Akinsanya]

### Task: Change LSTM model to Bidirectional LSTM Model， translate English to target language and evaluate using Bleu score.

### Due Date: Tuesday, April 19th, 11:59PM

## 0. You will do the following:

1. Read and run the code. Please make sure you have installed keras or tensorflow.Running the script on colab will speed up the training process and also prevent package loading issue. 
2. Implement the functions in Section 1 and 2, you may fill in your data directory.
3. Directly modify the code in Section 3. Change the current LSTM layer to a Bidirectional LSTM Model.
4. Training your model and translate English to Spanish in Section 4.2. You could try translating other languages.
5. Complete the code in Section 5.

## 1. Data preparation (10 points)

1. Download spanish-english data from http://www.manythings.org/anki/
2. You may try to use other languages.
3. Unzip the .ZIP file.
4. Put the .TXT file (e.g., "deu.txt") in the directory "./Data/".
5. Fill in your data directory in section 1.1.

### 1.1. Load and clean text


In [2]:
import re
import string
from unicodedata import normalize
import numpy

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text


# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

def clean_data(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
            line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
            line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
            line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return numpy.array(cleaned)

#### Fill the following blanks:

In [3]:
# e.g., filename = 'Data/deu.txt'
filename = "/content/spa.txt"

# e.g., n_train = 20000
n_train = 20000

In [4]:
# load dataset
doc = load_doc(filename)

# split into Language1-Language2 pairs
pairs = to_pairs(doc)
#print(len(pairs))

# clean sentences
clean_pairs = clean_data(pairs)[0:n_train, :]
test_pairs_1 = clean_data(pairs)[n_train:n_train+200,:]

In [5]:
for i in range(3000, 3010):
    print('[' + clean_pairs[i, 0] + '] => [' + clean_pairs[i, 1] + ']')

[youre here] => [estas aqui]
[youre here] => [estais aqui]
[youre late] => [estas retrasado]
[youre lost] => [estas perdido]
[youre mean] => [eres mala]
[youre mean] => [eres mezquino]
[youre mine] => [tu eres mio]
[youre nice] => [eres simpatico]
[youre nuts] => [estas loco]
[youre nuts] => [estas chiflado]


In [6]:
for i in range(30, 40):
    print('[' + test_pairs_1[i, 0] + '] => [' + test_pairs_1[i, 1] + ']')

[i have a flat tire] => [se me pincho una llanta]
[i have a flat tire] => [se me poncho una goma]
[i have a glass eye] => [tengo un ojo de cristal]
[i have a glass eye] => [tengo un ojo de vidrio]
[i have a list here] => [tengo aqui una lista]
[i have a nosebleed] => [me sangra la nariz]
[i have a toothache] => [estoy con dolor de dientes]
[i have a white cat] => [tengo un gato blanco]
[i have blonde hair] => [tengo el pelo rubio]
[i have cabin fever] => [tengo miedo a los lugares cerrados]


In [7]:
input_texts = clean_pairs[:, 0]
target_texts = ['\t' + text + '\n' for text in clean_pairs[:, 1]]

print('Length of input_texts:  ' + str(input_texts.shape))
print('Length of target_texts: ' + str(input_texts.shape))

Length of input_texts:  (20000,)
Length of target_texts: (20000,)


In [8]:
max_encoder_seq_length = max(len(line) for line in input_texts)
max_decoder_seq_length = max(len(line) for line in target_texts)

print('max length of input  sentences: %d' % (max_encoder_seq_length))
print('max length of target sentences: %d' % (max_decoder_seq_length))

max length of input  sentences: 18
max length of target sentences: 55


**Remark:** To this end, you have two lists of sentences: input_texts and target_texts

## 2. Text processing

### 2.1. Convert texts to sequences

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.

In [9]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# encode and pad sequences
def text2sequences(max_len, lines):
    tokenizer = Tokenizer(char_level=True, filters='')
    tokenizer.fit_on_texts(lines)
    seqs = tokenizer.texts_to_sequences(lines)
    seqs_pad = pad_sequences(seqs, maxlen=max_len, padding='post')
    return seqs_pad, tokenizer.word_index


encoder_input_seq, input_token_index = text2sequences(max_encoder_seq_length, 
                                                      input_texts)
decoder_input_seq, target_token_index = text2sequences(max_decoder_seq_length, 
                                                       target_texts)

print('shape of encoder_input_seq: ' + str(encoder_input_seq.shape))
print('shape of input_token_index: ' + str(len(input_token_index)))
print('shape of decoder_input_seq: ' + str(decoder_input_seq.shape))
print('shape of target_token_index: ' + str(len(target_token_index)))

shape of encoder_input_seq: (20000, 18)
shape of input_token_index: 27
shape of decoder_input_seq: (20000, 55)
shape of target_token_index: 29


In [10]:
num_encoder_tokens = len(input_token_index) + 1
num_decoder_tokens = len(target_token_index) + 1

print('num_encoder_tokens: ' + str(num_encoder_tokens))
print('num_decoder_tokens: ' + str(num_decoder_tokens))

num_encoder_tokens: 28
num_decoder_tokens: 30


**Remark:** To this end, the input language and target language texts are converted to 2 matrices. 

- Their number of rows are both n_train.
- Their number of columns are respective max_encoder_seq_length and max_decoder_seq_length.

The followings print a sentence and its representation as a sequence.

In [11]:
target_texts[100]

'\tno puede ser\n'

In [12]:
decoder_input_seq[100, :]

array([ 6,  8,  3,  1, 17, 14,  2, 15,  2,  1,  5,  2, 10,  7,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0], dtype=int32)

## 2.2. One-hot encode

- Input: A list of $n$ sentences (with max length $t$).
- It is represented by a $n\times t$ matrix after the tokenization and zero-padding.
- It is represented by a $n\times t \times v$ tensor ($t$ is the number of unique chars) after the one-hot encoding.

In [13]:
from tensorflow.keras.utils import to_categorical

# one hot encode target sequence
def onehot_encode(sequences, max_len, vocab_size):
    n = len(sequences)
    data = numpy.zeros((n, max_len, vocab_size))
    for i in range(n):
        data[i, :, :] = to_categorical(sequences[i], num_classes=vocab_size)
    return data

encoder_input_data = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
decoder_input_data = onehot_encode(decoder_input_seq, max_decoder_seq_length, num_decoder_tokens)

decoder_target_seq = numpy.zeros(decoder_input_seq.shape)
decoder_target_seq[:, 0:-1] = decoder_input_seq[:, 1:]
decoder_target_data = onehot_encode(decoder_target_seq, 
                                    max_decoder_seq_length, 
                                    num_decoder_tokens)

print(encoder_input_data.shape)
print(decoder_input_data.shape)

(20000, 18, 28)
(20000, 55, 30)


## 3. Build the networks (for training) (20 points)

- In this section, we have already implemented the LSTM model for you. You can run the code and see what the code is doing.  

- You need to change the existing LSTM model to a Bidirectional LSTM model. Just modify the network structrue and do not change the training cell in section 3.4.

- Build encoder, decoder, and connect the two modules to get "model". 

- Fit the model on the bilingual data to train the parameters in the encoder and decoder.



### 3.1. Encoder network

- Input:  one-hot encode of the input language

- Return: 

    -- output (all the hidden states   $h_1, \cdots , h_t$) are always discarded
    
    -- the final hidden state  $h_t$
    
    -- the final conveyor belt $c_t$

In [14]:
from tensorflow.keras.layers import Input, LSTM
from tensorflow.keras.models import Model

latent_dim = 256

# inputs of the encoder network
encoder_inputs = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs')

"""# set the LSTM layer
encoder_lstm = LSTM(latent_dim, return_state=True, 
                    dropout=0.5, name='encoder_lstm')
_, state_h, state_c = encoder_lstm(encoder_inputs)

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')"""

"# set the LSTM layer\nencoder_lstm = LSTM(latent_dim, return_state=True, \n                    dropout=0.5, name='encoder_lstm')\n_, state_h, state_c = encoder_lstm(encoder_inputs)\n\n# build the encoder network model\nencoder_model = Model(inputs=encoder_inputs, \n                      outputs=[state_h, state_c],\n                      name='encoder')"

In [15]:
from keras.layers import Bidirectional, Concatenate

encoder_bilstm = Bidirectional(LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm'))
_, forward_h, forward_c, backward_h, backward_c = encoder_bilstm(encoder_inputs)

state_h = Concatenate()([forward_h, backward_h])
state_c = Concatenate()([forward_c, backward_c])

# build the encoder network model
encoder_model = Model(inputs=encoder_inputs, 
                      outputs=[state_h, state_c],
                      name='encoder')

Print a summary and save the encoder network structure to "./encoder.pdf"

In [16]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(encoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model, show_shapes=False,
    to_file='encoder.pdf'
)

encoder_model.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs (InputLayer)    [(None, None, 28)]   0           []                               
                                                                                                  
 bidirectional (Bidirectional)  [(None, 512),        583680      ['encoder_inputs[0][0]']         
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                            

### 3.2. Decoder network

- Inputs:  

    -- one-hot encode of the target language
    
    -- The initial hidden state $h_t$ 
    
    -- The initial conveyor belt $c_t$ 

- Return: 

    -- output (all the hidden states) $h_1, \cdots , h_t$

    -- the final hidden state  $h_t$ (discarded in the training and used in the prediction)
    
    -- the final conveyor belt $c_t$ (discarded in the training and used in the prediction)

In [17]:
from keras.layers import Input, LSTM, Dense
from keras.models import Model
decoder_dim =512
# inputs of the decoder network
decoder_input_h = Input(shape=(decoder_dim,), name='decoder_input_h')
decoder_input_c = Input(shape=(decoder_dim,), name='decoder_input_c')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# set the LSTM layer
decoder_lstm = LSTM(decoder_dim, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm')
decoder_lstm_outputs, state_h, state_c = decoder_lstm(decoder_input_x, 
                                                      initial_state=[decoder_input_h, decoder_input_c])

# set the dense layer
decoder_dense = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense')
decoder_outputs = decoder_dense(decoder_lstm_outputs)

# build the decoder network model
decoder_model = Model(inputs=[decoder_input_x, decoder_input_h, decoder_input_c],
                      outputs=[decoder_outputs, state_h, state_c],
                      name='decoder')

Print a summary and save the encoder network structure to "./decoder.pdf"

In [18]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model, show_shapes=False,
    to_file='decoder.pdf'
)

decoder_model.summary()

Model: "decoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 decoder_input_h (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_input_c (InputLayer)   [(None, 512)]        0           []                               
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0][0]',        
                                 (None, 512),                     'decoder_input_h[0][0]',  

### 3.3. Connect the encoder and decoder

In [19]:
# input layers
encoder_input_x = Input(shape=(None, num_encoder_tokens), name='encoder_input_x')
decoder_input_x = Input(shape=(None, num_decoder_tokens), name='decoder_input_x')

# connect encoder to decoder
encoder_final_states = encoder_model([encoder_input_x])
decoder_lstm_output, _, _ = decoder_lstm(decoder_input_x, initial_state=encoder_final_states)
decoder_pred = decoder_dense(decoder_lstm_output)

model = Model(inputs=[encoder_input_x, decoder_input_x], 
              outputs=decoder_pred, 
              name='model_training')

In [20]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model, show_shapes=False,
    to_file='model_training.pdf'
)

model.summary()

Model: "model_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x (InputLayer)   [(None, None, 28)]   0           []                               
                                                                                                  
 decoder_input_x (InputLayer)   [(None, None, 30)]   0           []                               
                                                                                                  
 encoder (Functional)           [(None, 512),        583680      ['encoder_input_x[0][0]']        
                                 (None, 512)]                                                     
                                                                                                  
 decoder_lstm (LSTM)            [(None, None, 512),  1112064     ['decoder_input_x[0]

### 3.4. Fit the model on the bilingual dataset

- encoder_input_data: one-hot encode of the input language

- decoder_input_data: one-hot encode of the input language

- decoder_target_data: labels (left shift of decoder_input_data)

- tune the hyper-parameters

- stop when the validation loss stop decreasing.

In [21]:
print('shape of encoder_input_data' + str(encoder_input_data.shape))
print('shape of decoder_input_data' + str(decoder_input_data.shape))
print('shape of decoder_target_data' + str(decoder_target_data.shape))

shape of encoder_input_data(20000, 18, 28)
shape of decoder_input_data(20000, 55, 30)
shape of decoder_target_data(20000, 55, 30)


In [22]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

model.fit([encoder_input_data, decoder_input_data],  # training data
          decoder_target_data,                       # labels (left shift of the target sequences)
          batch_size=64, epochs=50, validation_split=0.2)

model.save('seq2seq.h5')

Epoch 1/50
250/250 [==============================] - 30s 83ms/step - loss: 0.9386 - val_loss: 0.8357
Epoch 2/50
250/250 [==============================] - 19s 76ms/step - loss: 0.7184 - val_loss: 0.7265
Epoch 3/50
250/250 [==============================] - 19s 76ms/step - loss: 0.6776 - val_loss: 0.6831
Epoch 4/50
250/250 [==============================] - 19s 76ms/step - loss: 0.6458 - val_loss: 0.6451
Epoch 5/50
250/250 [==============================] - 19s 75ms/step - loss: 0.6231 - val_loss: 0.6116
Epoch 6/50
250/250 [==============================] - 19s 75ms/step - loss: 0.6034 - val_loss: 0.5926
Epoch 7/50
250/250 [==============================] - 19s 75ms/step - loss: 0.5877 - val_loss: 0.5788
Epoch 8/50
250/250 [==============================] - 19s 75ms/step - loss: 0.5751 - val_loss: 0.5555
Epoch 9/50
250/250 [==============================] - 19s 75ms/step - loss: 0.5624 - val_loss: 0.5457
Epoch 10/50
250/250 [==============================] - 19s 76ms/step - loss: 0.551

## 4. Make predictions

- In this section, you need to complete section 4.2 to translate English to the target language.


### 4.1. Translate English to XXX

1. Encoder read a sentence (source language) and output its final states, $h_t$ and $c_t$.
2. Take the [star] sign "\t" and the final state $h_t$ and $c_t$ as input and run the decoder.
3. Get the new states and predicted probability distribution.
4. sample a char from the predicted probability distribution
5. take the sampled char and the new states as input and repeat the process (stop if reach the [stop] sign "\n").

In [23]:
# Reverse-lookup token index to decode sequences back to something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [24]:
def decode_sequence(input_seq):
    temperature = 0.2
    states_value = encoder_model.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = output_tokens[0, -1, :]
        sampled_token_index = sampled_token_index ** (1/temperature)
        sampled_token_index = sampled_token_index / numpy.sum(sampled_token_index)
        sampled_token_index = numpy.argmax(sampled_token_index)
        #print(sampled_token_index)
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


In [25]:
for seq_index in range(2100, 2120):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('English:       ', input_texts[seq_index])
    print('Spanish (true): ', target_texts[seq_index][1:-1])
    print('Spanish (pred): ', decoded_sentence[0:-1])


-
English:        hes skinny
Spanish (true):  el esta delgado
Spanish (pred):  el es mintio
-
English:        hes strong
Spanish (true):  el es fuerte
Spanish (pred):  el es a en proma
-
English:        hes stupid
Spanish (true):  el es estupido
Spanish (pred):  el es suy viejo
-
English:        hes stupid
Spanish (true):  no le llega agua al tanque
Spanish (pred):  el es suy viejo
-
English:        hes stupid
Spanish (true):  es un salame
Spanish (pred):  el es suy viejo
-
English:        help me out
Spanish (true):  ayudame
Spanish (pred):  ayudame a mary
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  ayudame a mary
-
English:        help me out
Spanish (true):  echeme la mano
Spanish (pred):  ayudame a mary
-
English:        help me out
Spanish (true):  ayudame a salir
Spanish (pred):  ayudame a mary
-
English:        here i come
Spanish (true):  aqui vengo
Spanish (pred):  aqui viene
-
English:        here i come
Spanish (true):  ya estoy aqui
Spani

### 4.2. Translate an English sentence to the target language （20 points）

1. Tokenization
2. One-hot encode
3. Translate

In [26]:
input_sentence = ['I love you']

input_sequence,dictr = text2sequences(max_encoder_seq_length, input_sentence)
print(input_sequence)
input_x = onehot_encode(input_sequence, max_encoder_seq_length, num_encoder_tokens)

translated_sentence = decode_sequence(input_x)

print('source sentence is: ' + str(input_sentence))
print('translated sentence is: ' + str(translated_sentence))

[[3 1 4 2 5 6 1 7 2 8 0 0 0 0 0 0 0 0]]
source sentence is: ['I love you']
translated sentence is: tom es un mentiroso



In [27]:
print(len(test_pairs_1))

200


In [28]:
###BLEU SCORE FOR THE FIRST MODEL

In [66]:
from nltk.translate.bleu_score import  sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
bleu_list = []
for n in range(len(test_pairs_1)):
  test_str = test_pairs_1[n][0]
  target_sentence = test_pairs_1[n][1]
  input_sentence = [test_str]
  encoder_input_seq,l= text2sequences(max_encoder_seq_length,input_sentence)
  input_x = onehot_encode(encoder_input_seq, max_encoder_seq_length, num_encoder_tokens)
  translated_sentence = decode_sequence(input_x)
  score = sentence_bleu(target_sentence, translated_sentence)
  #print(score)
  bleu_list.append(score)

print("Bleu_score ->", numpy.mean(bleu_list))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Bleu_score -> 0.7721547843360298


In [67]:
print('BLEU score is:',numpy.mean(bleu_list))


BLEU score is: 0.7721547843360298


# 5. Evaluate the translation using BLEU score

- We have already translated from English to target language, but how can we evaluate the performance of our model quantitatively? 

- In this section, you need to re-train the model we built in secton 3 and then evaluate the bleu score on testing dataset.

Reference:

https://machinelearningmastery.com/calculate-bleu-score-for-text-python/

https://en.wikipedia.org/wiki/BLEU

#### Hint:

- Randomly partition the dataset to training, validation, and test.

- Evaluate the BLEU score using the test set. Report the average.

- You may use packages to calculate bleu score, e.g., sentence_bleu() from nltk package.

[link text](https://)### 5.1. Partition the dataset to training, validation, and test. Build new token index. (10 points)

1. You may try to load more data/lines from text file.
2. Convert text to sequences and build token index using training data.
3. One-hot encode your training and validation text sequences.

In [33]:
#Load the whole dataset again from scratch

In [34]:
clean_pairs_2 = clean_data(pairs)[0:60200, :]

In [35]:
data_length = len(clean_pairs_2)
print(data_length)

60200


In [36]:
numpy.random.seed(42)
random_indices = numpy.random.permutation(data_length)
test_pairs_indices = random_indices[:200]
training_pairs_indices = random_indices[200:]

In [37]:
test_pairs = clean_pairs_2[test_pairs_indices]
print(len(test_pairs))

200


In [38]:
training_pairs = clean_pairs_2[training_pairs_indices]
print(len(training_pairs))

60000


In [39]:
input_texts_2 = training_pairs[:, 0]
target_texts_2 = ['\t' + text + '\n' for text in training_pairs[:, 1]]

print('Length of input_texts_2:  ' + str(input_texts_2.shape))
print('Length of target_texts_2: ' + str(input_texts_2.shape))

Length of input_texts_2:  (60000,)
Length of target_texts_2: (60000,)


In [40]:
max_encoder_seq_length_2 = max(len(line) for line in input_texts_2)
max_decoder_seq_length_2 = max(len(line) for line in target_texts_2)

print('max length of input  sentences: %d' % (max_encoder_seq_length_2))
print('max length of target sentences: %d' % (max_decoder_seq_length_2))

max length of input  sentences: 26
max length of target sentences: 68


In [41]:
encoder_input_seq_2, input_token_index_2 = text2sequences(max_encoder_seq_length_2, 
                                                      input_texts_2)
decoder_input_seq_2, target_token_index_2 = text2sequences(max_decoder_seq_length_2, 
                                                       target_texts_2)

print('shape of encoder_input_seq_2: ' + str(encoder_input_seq_2.shape))
print('shape of input_token_index_2: ' + str(len(input_token_index_2)))
print('shape of decoder_input_seq_2: ' + str(decoder_input_seq_2.shape))
print('shape of target_token_index_2: ' + str(len(target_token_index_2)))

shape of encoder_input_seq_2: (60000, 26)
shape of input_token_index_2: 27
shape of decoder_input_seq_2: (60000, 68)
shape of target_token_index_2: 29


In [42]:
#One-hot encode

In [43]:
encoder_input_data_2 = onehot_encode(encoder_input_seq_2, max_encoder_seq_length_2, num_encoder_tokens)
decoder_input_data_2 = onehot_encode(decoder_input_seq_2, max_decoder_seq_length_2, num_decoder_tokens)

decoder_target_seq_2 = numpy.zeros(decoder_input_seq_2.shape)
decoder_target_seq_2[:, 0:-1] = decoder_input_seq_2[:, 1:]
decoder_target_data_2 = onehot_encode(decoder_target_seq_2, 
                                    max_decoder_seq_length_2, 
                                    num_decoder_tokens)

print(encoder_input_data_2.shape)
print(decoder_input_data_2.shape)

(60000, 26, 28)
(60000, 68, 30)


In [44]:
latent_dim = 256

# inputs of the encoder network
encoder_inputs_2 = Input(shape=(None, num_encoder_tokens), 
                       name='encoder_inputs_2')

In [45]:
encoder_bilstm_2 = Bidirectional(LSTM(latent_dim, return_state=True, 
                                  dropout=0.5, name='encoder_lstm_2'))
_, forward_h_2, forward_c_2, backward_h_2, backward_c_2 = encoder_bilstm_2(encoder_inputs_2)

state_h_2 = Concatenate()([forward_h_2, backward_h_2])
state_c_2 = Concatenate()([forward_c_2, backward_c_2])

# build the encoder network model
encoder_model_2 = Model(inputs=encoder_inputs_2, 
                      outputs=[state_h_2, state_c_2],
                      name='encoder_2')

In [46]:
SVG(model_to_dot(encoder_model_2, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=encoder_model_2, show_shapes=False,
    to_file='encoder_2.pdf'
)

encoder_model_2.summary()

Model: "encoder_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_inputs_2 (InputLayer)  [(None, None, 28)]   0           []                               
                                                                                                  
 bidirectional_1 (Bidirectional  [(None, 512),       583680      ['encoder_inputs_2[0][0]']       
 )                               (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256),                                                     
                                 (None, 256)]                                                     
                                                                                          

In [47]:
decoder_dim_2 =512
# inputs of the decoder network
decoder_input_h_2 = Input(shape=(decoder_dim_2,), name='decoder_input_h_2')
decoder_input_c_2 = Input(shape=(decoder_dim_2,), name='decoder_input_c_2')
decoder_input_x_2 = Input(shape=(None, num_decoder_tokens), name='decoder_input_x_2')

# set the LSTM layer
decoder_lstm_2 = LSTM(decoder_dim_2, return_sequences=True, 
                    return_state=True, dropout=0.5, name='decoder_lstm_2')
decoder_lstm_outputs_2, state_h_2, state_c_2 = decoder_lstm_2(decoder_input_x_2, 
                                                      initial_state=[decoder_input_h_2, decoder_input_c_2])

# set the dense layer
decoder_dense_2 = Dense(num_decoder_tokens, activation='softmax', name='decoder_dense_2')
decoder_outputs_2 = decoder_dense_2(decoder_lstm_outputs_2)

# build the decoder network model
decoder_model_2 = Model(inputs=[decoder_input_x_2, decoder_input_h_2, decoder_input_c_2],
                      outputs=[decoder_outputs_2, state_h_2, state_c_2],
                      name='decoder_2')

In [48]:
### Print a summary and save the encoder network structure to "./decoder.pdf"

In [49]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(decoder_model_2, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=decoder_model_2, show_shapes=False,
    to_file='decoder_2.pdf'
)

decoder_model_2.summary()

Model: "decoder_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 decoder_input_x_2 (InputLayer)  [(None, None, 30)]  0           []                               
                                                                                                  
 decoder_input_h_2 (InputLayer)  [(None, 512)]       0           []                               
                                                                                                  
 decoder_input_c_2 (InputLayer)  [(None, 512)]       0           []                               
                                                                                                  
 decoder_lstm_2 (LSTM)          [(None, None, 512),  1112064     ['decoder_input_x_2[0][0]',      
                                 (None, 512),                     'decoder_input_h_2[0][0]

In [50]:
### 3.3. Connect the encoder and decoder

In [51]:
# input layers
encoder_input_x_2 = Input(shape=(None, num_encoder_tokens), name='encoder_input_x_2')
decoder_input_x_2 = Input(shape=(None, num_decoder_tokens), name='decoder_input_x_2')

# connect encoder to decoder
encoder_final_states_2 = encoder_model_2([encoder_input_x_2])
decoder_lstm_output_2, _, _ = decoder_lstm_2(decoder_input_x_2, initial_state=encoder_final_states_2)
decoder_pred_2 = decoder_dense_2(decoder_lstm_output_2)

model_2 = Model(inputs=[encoder_input_x_2, decoder_input_x_2], 
              outputs=decoder_pred_2, 
              name='model_2_training')

In [52]:
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot, plot_model

SVG(model_to_dot(model_2, show_shapes=False).create(prog='dot', format='svg'))

plot_model(
    model=model_2, show_shapes=False,
    to_file='model_2_training.pdf'
)

model_2.summary()

Model: "model_2_training"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 encoder_input_x_2 (InputLayer)  [(None, None, 28)]  0           []                               
                                                                                                  
 decoder_input_x_2 (InputLayer)  [(None, None, 30)]  0           []                               
                                                                                                  
 encoder_2 (Functional)         [(None, 512),        583680      ['encoder_input_x_2[0][0]']      
                                 (None, 512)]                                                     
                                                                                                  
 decoder_lstm_2 (LSTM)          [(None, None, 512),  1112064     ['decoder_input_x_

In [53]:
### 3.4. Fit the model on the bilingual dataset


In [54]:
print('shape of encoder_input_data_2' + str(encoder_input_data_2.shape))
print('shape of decoder_input_data_2' + str(decoder_input_data_2.shape))
print('shape of decoder_target_data_2' + str(decoder_target_data_2.shape))

shape of encoder_input_data_2(60000, 26, 28)
shape of decoder_input_data_2(60000, 68, 30)
shape of decoder_target_data_2(60000, 68, 30)


In [ ]:
### Decode Sequence 2 function for second model Built

In [55]:
def decode_sequence_2(input_seq):
    states_value = encoder_model_2.predict(input_seq)

    target_seq = numpy.zeros((1, 1, num_decoder_tokens))
    target_seq[0, 0, target_token_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model_2.predict([target_seq] + states_value)

        # this line of code is greedy selection
        # try to use multinomial sampling instead (with temperature)
        sampled_token_index = numpy.argmax(output_tokens[0, -1, :])
        
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        target_seq = numpy.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        states_value = [h, c]

    return decoded_sentence


### 5.2 Retrain your previous Bidirectional LSTM model with training and validation data and tune the parameters (learning rate, optimizer, etc) based on validation score. (25 points)

1. Use the model structure in section 3 to train a new model with new training and validation datasets.
2. Based on validation BLEU score or loss to tune parameters.

In [56]:
from keras import optimizers

model_2.compile(optimizer=optimizers.rmsprop_v2.RMSprop(learning_rate=0.01), loss='categorical_crossentropy')

model_2.fit([encoder_input_data_2, decoder_input_data_2],  # training data
          decoder_target_data_2,                       # labels (left shift of the target sequences)
          batch_size=32, epochs=10, validation_split=0.2)

model_2.save('seq2seq_2.h5')

Epoch 1/5
1500/1500 [==============================] - 128s 83ms/step - loss: 0.8031 - val_loss: 0.5233
Epoch 2/5
1500/1500 [==============================] - 122s 82ms/step - loss: 0.6420 - val_loss: 0.4686
Epoch 3/5
1500/1500 [==============================] - 124s 82ms/step - loss: 0.6110 - val_loss: 0.4559
Epoch 4/5
1500/1500 [==============================] - 122s 82ms/step - loss: 0.5940 - val_loss: 0.4518
Epoch 5/5
1500/1500 [==============================] - 124s 83ms/step - loss: 0.5830 - val_loss: 0.4497


### 5.3 Evaluate the BLEU score using the test set. (15 points)

1. Use trained model above to calculate the BLEU score with testing dataset.

In [63]:
from nltk.translate.bleu_score import  sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction
smoothie = SmoothingFunction().method1
bleu_list_2 = []
for n in range(len(test_pairs)):
  test_str = test_pairs[n][0]
  target_sentence = test_pairs[n][1]
  input_sentence = [test_str]
  encoder_input_seq,l= text2sequences(max_encoder_seq_length_2,input_sentence)
  input_x = onehot_encode(encoder_input_seq, max_encoder_seq_length_2, num_encoder_tokens)
  try:
    translated_sentence = decode_sequence_2(input_x)
  except KeyError as err:
    continue
  score = sentence_bleu(target_sentence, translated_sentence)
  #print(score)
  bleu_list_2.append(score)

print("Bleu_score ->", numpy.mean(bleu_list_2))

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


Bleu_score -> 0.7781658255534205


In [64]:
print(len(bleu_list_2),'BLEU score is:',numpy.mean(bleu_list_2))


200 BLEU score is: 0.7781658255534205
